In [89]:
# Ελεγχος version pyspark .. Θέλουμε το 3.4.1
import pyspark
print(pyspark.__version__)

3.4.1


In [90]:
# add java to path .. δεν μπορουσα να βρω αλλον τροπο να τρέξει η spark.
# (θελουμε την Java 17 επειδη μονο η Java 8 , Java 11 και Java 17 δουλευει με το pyspark)
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"
os.environ["PATH"] += ";C:\\Program Files\\Java\\jdk-17\\bin"

In [91]:
from pyspark.sql import SparkSession
import pandas as pd

In [92]:
# Διαβάζομε τα δεδομένα

tour_occ_ninat = "data/tour_occ_ninat.xlsx"

excel_data = pd.read_excel(tour_occ_ninat, header=8)

In [93]:
excel_data.to_csv("data/excel_data.csv", index=False)

                                            GEO/TIME       2006       2007       2008       2009       2010       2011       2012       2013       2014       2015
0                                            Belgium   16039090   16271311   16360702   15451017   16169676   16723867   16432646   16511721   17068872   18852087
1                                           Bulgaria   11944694   12006786   11791454    9460922   10547112   12426723   13451440   14370426   14077798   13352281
2                                     Czech Republic   20090348   20610186   19987022   17746893   18365947   19424839   21793985   22144896   22110112   23286515
3                                            Denmark    9453026    9327579    8918197    8299403    8981992    9491137    9608124    9914273   10608119   11171416
4   Germany (until 1990 former territory of the FRG)   52947373   54485379   56239679   54096574   59658760   63081467   68161503   71191942   74805253   78827773
5                     

In [94]:
# Φτιάχνουμε ενα spark session
spark = SparkSession.builder.appName("EurostatAnalysis").getOrCreate()

In [95]:
# Φτιάχνουμε ενα spark dataframe
spark_df = spark.read.option("header", "true").csv("data/excel_data.csv")


In [96]:
spark_df.show(truncate=False)

+------------------------------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|GEO/TIME                                        |2006     |2007     |2008     |2009     |2010     |2011     |2012     |2013     |2014     |2015     |
+------------------------------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|Belgium                                         |16039090 |16271311 |16360702 |15451017 |16169676 |16723867 |16432646 |16511721 |17068872 |18852087 |
|Bulgaria                                        |11944694 |12006786 |11791454 |9460922  |10547112 |12426723 |13451440 |14370426 |14077798 |13352281 |
|Czech Republic                                  |20090348 |20610186 |19987022 |17746893 |18365947 |19424839 |21793985 |22144896 |22110112 |23286515 |
|Denmark                                         |9453026  |9327579  |8918197  |8299403  |8981